In [193]:
import pandas as pd
import sys
from statistics import median
import pprint

In [176]:
sbs_result_file = '../RQ1/results/gfm_9_final_result.csv'
RQ2_result_file = '../RQ2/final_gfm_ssr_results.csv'
RQ3_result_root = '../RQ3/final_results/'
old_result_file = 'New_Updated_RQ_Results.csv'
ssr_delay_file = '../RQ2/trial.csv'
sbs_delay_file = '../RQ1/final_sbs_results/sbs_median_delays.csv'
ml_delay_file = '../RQ3/final_results/ml_delay_list.csv'

In [177]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']

In [178]:
old = pd.read_csv(old_result_file, index_col=0)
gr_results = old[ (old['method']=='gr') & (old['batch_size'] != 32) & ( old['batch_size'] != 1 )]
projects = set(gr_results['project'])

In [179]:
final_results = gr_results

In [180]:
sbs = pd.read_csv(sbs_result_file, index_col=0)
sbs_delay = pd.read_csv(sbs_delay_file, index_col=0).to_dict('index')

In [181]:
for i in list(sbs_delay.keys()):
    sbs_delay[sbs_delay[i]['project']+'.csv'] = { 'delay_list':sbs_delay[i]['delay_list'], 'median_delay':sbs_delay[i]['median_delay']}
    sbs_delay.pop(i)

In [182]:
sbs = sbs[sbs['batch']==1]
for p in range(len(sbs)):
    p_data = sbs.iloc[p]
    for b in batch_sizes:
        for alg in batching_alg:
            if alg == 'BATCH4':
                if b != 4:
                    continue
            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue
            
            new_row = [p_data.name, 'sbs', alg, b, p_data[2], 0, sbs_delay[p_data.name]['median_delay']]
            
            final_results.loc[len(final_results.index)] = new_row

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [183]:
ssr = pd.read_csv(RQ2_result_file, usecols=['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays'])
ssr_delay = pd.read_csv(ssr_delay_file)

In [184]:
for x in range(len(ssr)):
    row = ssr.iloc[x]
    val = ssr_delay[ (ssr_delay['project']==row['project']) & (ssr_delay['batch_size']==row['batch_size']) & (ssr_delay['algorithm']==row['algorithm'])]
    s = val['sbs_delays'].values[0]
    if s == '[]':
        row['sbs_delays'] = [0]
    else:
        l = s[1:-1].split(', ')
        l = [int(x) for x in l]
        row['sbs_delays'] = median(l)
    final_results.loc[len(final_results.index)] = row
    

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


In [185]:
final_results[ final_results['method'] == 'ssr']

,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays
304.0,loomio.csv,ssr,BATCHSTOP4,4,37.560974,150.0,7.0
305.0,loomio.csv,ssr,BATCHSTOP4,8,33.658535,364.0,9.5
306.0,loomio.csv,ssr,BATCHSTOP4,16,58.536587,1171.0,14.0
307.0,loomio.csv,ssr,BATCH4,4,37.560974,150.0,7.0
308.0,loomio.csv,ssr,BATCHBISECT,1,38.536587,0.0,3.0
...,...,...,...,...,...,...,...
470.0,open-build-service.csv,ssr,BATCHBISECT,1,37.553650,0.0,3.0
471.0,open-build-service.csv,ssr,BATCHBISECT,2,40.772533,134.0,2.5
472.0,open-build-service.csv,ssr,BATCHBISECT,4,29.399141,261.0,5.5
473.0,open-build-service.csv,ssr,BATCHBISECT,8,37.768242,899.0,7


In [186]:
ml_delay = pd.read_csv(ml_delay_file, index_col=0)

In [187]:
ml_lib = {}
for i in range(len(ml_delay)):
    r = ml_delay.iloc[i]
    if r['project']+'.csv' not in ml_lib:
        ml_lib[r['project']+'.csv'] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{4:{}, 2:{}, 8:{}, 16:{}}}
        ml_lib[r['project']+'.csv'][r['alg']][r['batch_size']]['median_delay'] = r['median_delay']
    else:
        ml_lib[r['project']+'.csv'][r['alg']][r['batch_size']]['median_delay'] = r['median_delay']
    

In [188]:
lines = []
for p in projects:
    filename = '../RQ3/final_results/'+ p.split('.')[0] + '_ml_batching.csv'
    p_data = pd.read_csv(filename, usecols=['project', 'algorithm', 'batch_size', 'builds_reqd', 'total_delay', 'batch_delays'])
    for k in range(len(p_data)):
        row = p_data.iloc[k]
        final_row = [row[0], 'ml', row[1], row[2], row[3], row[5], ml_lib[row[0]][row[1]][row[2]]['median_delay']]
        lines.append(final_row)        

In [189]:
df = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays'])

In [190]:
final_results = final_results.append(df)

In [191]:
final_results

,project,method,algorithm,batch_size,builds_reqd,batch_delays,sbs_delays
759.0,cloud_controller_ng.csv,gr,BATCHBISECT,2,76.284585,127.0,0.0
778.0,cloud_controller_ng.csv,gr,BATCHBISECT,4,76.679842,381.0,0.0
797.0,cloud_controller_ng.csv,gr,BATCH4,4,75.889328,381.0,0.0
816.0,cloud_controller_ng.csv,gr,BATCHSTOP4,4,75.889328,381.0,0.0
835.0,cloud_controller_ng.csv,gr,BATCHBISECT,8,82.213439,893.0,0.0
...,...,...,...,...,...,...,...
147.0,rails.csv,ml,BATCHSTOP4,16,66.892349,11040.0,1.0
148.0,rails.csv,ml,BATCHBISECT,2,60.257278,634.0,1.0
149.0,rails.csv,ml,BATCHBISECT,4,60.392688,2076.0,1.0
150.0,rails.csv,ml,BATCHBISECT,8,64.251862,5068.0,2.0


In [192]:
final_results.to_csv('median_delay_all_methods.csv')